In [32]:
# download preprocessed_train
import pandas as pd

train = pd.read_csv("/content/preprocessed_train.csv", index_col=0)
train

,sentiment,tweet,length
664377,0.0,cook supper miss workout due inabl breath bodi...,80.0
1069181,1.0,wee star girl aloud not vote julian must cast ...,67.0
204198,0.0,ffxingi want banh mi nguyen ngo,31.0
828039,1.0,go bed,6.0
827562,1.0,happi birthday mr b,19.0
...,...,...,...
172761,0.0,dude what wana go,17.0
719247,0.0,slept hr iran crazytown went bed much happen,44.0
373729,0.0,wish danc friend live illinoi,29.0
308813,0.0,wuaaahmiss dcinema much mani movi ive missedhuh,47.0


In [33]:
# Why we have NA????!!!

train.dropna(inplace=True)

In [34]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(train, test_size=0.2, random_state=42)

In [35]:
### Froooooooooooooom heeeeeeeeeeeeeeeere
import gensim.downloader as api

word2vec_model = api.load("glove-twitter-25")  # load glove vectors


In [36]:
X_train, y_train = train["tweet"], train["sentiment"]
X_val, y_val = val["tweet"], val["sentiment"]

In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [38]:
X_train_tokenized = tokenizer.texts_to_sequences(X_train)
X_val_tokenized = tokenizer.texts_to_sequences(X_val)

In [39]:
X_train_tokenized

[[7733, 10116, 24, 17, 832, 29, 120, 7734, 6902, 53],
 [2163, 7735, 30, 542, 141, 1154, 1, 78, 2, 1154, 2599],
 [228, 5401, 7, 446, 1069, 52, 271, 447, 1119, 551],
 [4862, 177, 109],
 [36, 45],
 [557, 1498, 122, 31, 1861, 29],
 [52,
  24163,
  453,
  58,
  18,
  5632,
  419,
  2324,
  24,
  24164,
  545,
  2180,
  492,
  453,
  205,
  146,
  1965,
  24165],
 [25, 1851, 114],
 [141, 2741, 1362, 418],
 [338,
  24166,
  1887,
  8,
  153,
  221,
  765,
  1887,
  223,
  16662,
  24167,
  16663,
  5027,
  34,
  1410,
  838],
 [163, 1, 601, 155, 1, 43, 65, 472, 765, 202, 115, 197],
 [195, 383, 975, 439, 85, 36, 7, 334, 391, 109],
 [5, 25],
 [380],
 [152, 5, 1482, 177, 203, 2475, 48, 810, 3442],
 [5],
 [1420, 1748, 55, 364, 476, 12, 3142, 340, 76, 24168, 34, 7736],
 [146, 1049, 1, 315, 5894],
 [290, 39, 7, 398, 48, 25, 86, 786, 1229, 108, 776, 50, 293, 37, 2],
 [181, 24169, 24170, 24171, 164, 24172, 7, 9, 1396, 67, 2535],
 [303, 280],
 [2, 26, 5895, 13381],
 [6903, 956, 13382],
 [69, 804, 1345

In [40]:
# unique values
len(tokenizer.word_index)

73577

In [41]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAXLEN = 20
X_train_tokenized_padded = pad_sequences(X_train_tokenized, maxlen=MAXLEN, padding='post')
X_val_tokenized_padded = pad_sequences(X_val_tokenized, maxlen=MAXLEN, padding='post')

In [42]:
X_train_tokenized_padded

array([[ 7733, 10116,    24, ...,     0,     0,     0],
       [ 2163,  7735,    30, ...,     0,     0,     0],
       [  228,  5401,     7, ...,     0,     0,     0],
       ...,
       [  802,  1640,     0, ...,     0,     0,     0],
       [  293,   531,  1118, ...,     0,     0,     0],
       [  440,     0,     0, ...,     0,     0,     0]], dtype=int32)

##build embedding from scratch

In [52]:
from tensorflow.keras.preprocessing.text import Tokenizer # Import Tokenizer from the correct module
tokenizer = Tokenizer()

In [53]:
import numpy as np
import tensorflow as tf

embedding_matrix = np.zeros((Vocab_size, Embedding_size))

for word, idx in tokenizer.word.items():
  if word in word2vec_model:
    embedding_matrix[idx] = word2vec_model[word]

embedding_matrix

AttributeError: 'Tokenizer' object has no attribute 'word'

In [48]:
from tensorflow.keras.layers import Embedding, Dense, Flatten, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential


Vocab_size = 175959 + 2 # unique words + 1 paddding + 1 UNK
Embedding_size = 25

model = Sequential([Embedding(input_dim=Vocab_size,
                              output_dim=Embedding_size,
                              embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix), # init randow weight   before None
                              input_shape=(MAXLEN, ) ), # 2D
                    # GlobalAveragePooling1D(), # 2D to # 1D
                    SimpleRNN(units=50, activation="tanh"),

                    # Flatten(), #2D t0 1D
                    Dense(units=100, activation="relu"),
                    Dense(units=1, activation="sigmoid")]) # 1D

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

# fit
# model.fit(X_train_tokenized_padded, y_train, epochs=5, validation_data=(X_val_tokenized_padded, y_val))

model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)              │ (None, 20, 25)              │       4,399,025 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ (None, 25)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 100)                 │           2,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,401,726 (16.79 MB)

 Trainable params: 4,401,726 (16.79 MB)

 Non-trainable params: 0 (0.00 B)